In [ ]:
import numpy as np
import pandas as pd
import os
import umap
import matplotlib.pyplot as plt
import seaborn as sb
import scipy.linalg

In [ ]:
# Load features from population.csv file
population_df = pd.read_csv('/raid/data/PUMA/cdr/population_normalized.csv')

In [ ]:
#Load broad compound ids which are used in the current PUMA experiments
broad_ids_df = pd.read_csv('broad_ids.txt', header = None)
broad_ids = broad_ids_df.loc[:,0].to_list()
len(broad_ids)

In [ ]:
# Filter population.csv, leave only compounds from PUMA experiment + DMSO
population_df_filtered = population_df[ (population_df["Metadata_broad_sample"] == "DMSO") | (population_df["Metadata_pert_id"].isin(broad_ids)) ].reset_index(drop=True).copy()
population_df_filtered

In [ ]:
#Get lists of feature column names
feature_columns = population_df_filtered.columns[20:].tolist()
nan_columns = []
for i in feature_columns:
    if population_df_filtered[i].isnull().values.any():
        nan_columns.append(i)

feature_columns = list(set(feature_columns) - set(nan_columns))

In [ ]:
class WhiteningNormalizer(object):
    def __init__(self, controls):
        REG_PARAM = 10**np.log(1/controls.shape[0])
        # Whitening transform on population level data
        self.mu = controls.mean()
        self.whitening_transform(controls - self.mu, REG_PARAM, rotate=True)
        print(self.mu.shape, self.W.shape)
        
    def whitening_transform(self, X, lambda_, rotate=True):
        C = (1/X.shape[0]) * np.dot(X.T, X)
        s, V = scipy.linalg.eigh(C)
        D = np.diag( 1. / np.sqrt(s + lambda_) )
        W = np.dot(V, D)
        if rotate:
            W = np.dot(W, V.T)
        self.W = W

    def normalize(self, X):
        return np.dot(X - self.mu, self.W)

In [ ]:
#Start whitening
whN = WhiteningNormalizer(population_df_filtered.loc[population_df_filtered["Metadata_broad_sample"] == "DMSO", feature_columns])
whD = whN.normalize(population_df_filtered[feature_columns])

In [ ]:
#Replace original feature values with feature values after whitening
population_df_filtered[feature_columns] = whD

In [ ]:
#Mean aggregation
aggregated_whitened = population_df_filtered[['Metadata_broad_sample', 'Metadata_Plate_Map_Name', 'Metadata_pert_id'] + feature_columns].copy()
aggregated_whitened = aggregated_whitened.groupby("Metadata_broad_sample").mean().reset_index()

aggregated_whitened_np = aggregated_whitened[aggregated_whitened['Metadata_broad_sample'] != 'DMSO'].to_numpy()

In [ ]:
# Save features. Those should be preprocessed later (sorted in the same way as in other experiments, remove first column)
np.savez('aggregated_whitened_morphology_features_norm', features=aggregated_whitened_np)

In [ ]:
#Get UMAP plot of aggregated features after whitening
reducer = umap.UMAP()
embeddings = reducer.fit_transform(aggregated_whitened_np[:,1:])
plt.figure(figsize=(10,10))
plt.scatter(x=embeddings[:,0], y=embeddings[:,1])

In [ ]:
# Make a dataframe of UMAP embeddings
embeddings = np.concatenate((embeddings, np.reshape(aggregated_whitened_np[:,0],(aggregated_whitened_np[:,0].size, 1))), axis=1)
embeddings_df = pd.DataFrame(embeddings, columns = ['X', 'Y', 'Metadata_broad_sample']) 

In [ ]:
# Merge embeddings dataframe with other metadata
temp = population_df_filtered[['Metadata_broad_sample', 'Metadata_Plate_Map_Name', 'Metadata_pert_id']].copy()
temp = temp.groupby(['Metadata_broad_sample', 'Metadata_Plate_Map_Name', 'Metadata_pert_id'], as_index=False).size().reset_index(name = "Count").drop(columns = ["Count"])
embeddings_full_df = pd.merge(embeddings_df.reset_index(drop=True), temp , on="Metadata_broad_sample", how="left")

In [ ]:
embeddings_full_df.to_csv('aggregated_umap_python_whitening_norm.csv')